Connected to my_env (Python 3.9.18)

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
df = pd.read_csv("/gpfs/Labs/Uzun/SCRIPTS/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER/output/DS011_mESC/DS011_mESC_sample1/ds011_nite_predictions.csv", header=0, index_col=0)

In [ ]:
df

In [ ]:
nite_predictions = df.mean(axis=1)
nite_predictions

In [ ]:
nite_predictions = df.mean(axis=1).rename_axis("NITE_score")
nite_predictions

In [ ]:
nite_predictions = df.mean(axis=1).rename("NITE_score")
nite_predictions

In [ ]:
nite_predictions = nite_predictions.to_frame()

In [ ]:
nite_predictions

In [ ]:
nite_predictions.to_parquet("output/DS011_mESC/DS011_mESC_sample1/ds011_nite_predictions.parquet", engine="pyarrow", compression="snappy")

In [ ]:
mira_df = pd.read_parquet("output/DS011_mESC/DS011_mESC_sample1/ds011_peak_to_gene_lite_rp_score_chrom_diff.parquet")

In [ ]:
mira_df

In [ ]:
nite_predictions

In [ ]:
nite_df = nite_predictions.rename_axis("target_id", axis=1)

In [ ]:
nite_df

In [ ]:
nite_df = nite_predictions.reset_index()

In [ ]:
nite_df

In [ ]:
nite_df = nite_df.rename(columns={"index": "target_id"})

In [ ]:
nite_df

In [ ]:
mira_df

In [ ]:
merged_df = pd.merge(mira_df, nite_df, on="target_id", how="inner")

In [ ]:
merged_df

In [ ]:
merged_df = merged_df.rename(columns={"MIRA_LITE_RP_score": "LITE_score"})
merged_df = merged_df[["peak_id", "target_id", "LITE_score", "NITE_score", "chromatin_differential"]]

In [ ]:
merged_df

In [ ]:
merged_df.to_parquet("output/DS011_mESC/DS011_mESC_sample1/ds011_peak_to_gene_lite_rp_score_chrom_diff.parquet", engine="pyarrow", compression="snappy")

In [ ]:
print(merged_df.head())

In [ ]:
score_df = pd.read_parquet("output/DS011_mESC/DS011_mESC_sample1/labeled_inferred_grn.parquet")

In [ ]:
score_df

In [ ]:
score_cols = [
    'mean_tf_expr', 'mean_peak_expr', 'mean_tg_expr',
    'cicero_score', 'TSS_dist_score', 'correlation',
    'LITE_score', 'NITE_score', 'chromatin_differential', 
    'sliding_window_score', 'homer_binding_score',
    ]


corrs = score_df[score_cols + ["label"]].corr()["label"].sort_values(ascending=False)
print(corrs)

In [ ]:
from sklearn.feature_selection import mutual_info_classif
mi = mutual_info_classif(score_df[score_cols], score_df["label"])
print(pd.Series(mi, index=score_cols).sort_values(ascending=False))

In [ ]:
from sklearn.feature_selection import mutual_info_classif
import pandas as pd

# Subset relevant features and label
cols = score_cols + ['label']
df_subset = score_df[cols].copy()

# Drop rows with any NaNs (safe for quick analysis)
df_subset = df_subset.dropna()

# Separate X (features) and y (labels)
X = df_subset[score_cols]
y = df_subset['label']

# Compute mutual information
mi = mutual_info_classif(X, y, discrete_features=False, random_state=42)
mi_series = pd.Series(mi, index=score_cols).sort_values(ascending=False)

print(mi_series)

In [ ]:
score_df = pd.read_parquet("output/DS011_mESC/DS011_mESC_sample1/labeled_inferred_grn.parquet")

In [ ]:
score_df

In [ ]:
from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

score_df = pd.read_parquet("output/DS011_mESC/DS011_mESC_sample1/labeled_inferred_grn.parquet")

score_cols = [
    'mean_tf_expr', 'mean_peak_expr', 'mean_tg_expr',
    'cicero_score', 'TSS_dist_score', 'correlation',
    'LITE_score', 'NITE_score', 'chromatin_differential', 
    'sliding_window_score', 'homer_binding_score', 'string_experimental_score',
    'string_textmining_score', 'string_combined_score'
    ]

results = []

# Iterate through each TF
for tf, group in score_df.groupby("source_id"):
    X = group[score_cols].fillna(0).copy()
    y = group["label"].fillna(0).copy()

    # Skip if too few samples
    if len(X) < 1 or y.nunique() < 2:
        continue

    mi_scores = mutual_info_classif(X, y, random_state=42, discrete_features=False)
    for feat, score in zip(score_cols, mi_scores):
        results.append({"TF": tf, "Feature": feat, "MI_score": score})

# Combine results
mi_df = pd.DataFrame(results)


pivot = mi_df.pivot(index="TF", columns="Feature", values="MI_score")
plt.figure(figsize=(14, len(pivot) * 0.8))
sns.heatmap(pivot, cmap="viridis", linewidths=0)
plt.title("Mutual Information per TF")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()